In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # checkpointer to implement persistence
# it stores all the values in RAM

/home/rishukumar/Documents/LangGraph_CampusX/myenv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

E0000 00:00:1760127904.438604    7931 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza confess its deepest feelings?\n\nBecause it was feeling a little **cheesy**!',
 'explanation': 'This joke is a classic pun, playing on the double meaning of the word **"cheesy"**.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Obviously, pizza is made with cheese. So, in a literal sense, a pizza is *always* "cheesy" because it contains cheese. This is the setup for the joke.\n\n2.  **Figurative Meaning (Feelings/Behavior):** The word "cheesy" is also used to describe something that is overly sentimental, sappy, corny, or trying too hard to be romantic or emotional. When someone expresses their "deepest feelings" in an overly dramatic or sentimental way, we often say it\'s "cheesy" (e.g., "That\'s a cheesy pick-up line," or "That love song is a bit cheesy").\n\nThe humor comes from the unexpected link between these two meanings. The pizza, being literally full of cheese, is also metaphorically "cheesy" in its emotional expres

In [8]:
workflow.get_state(config1) # it will give me the final state value

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza confess its deepest feelings?\n\nBecause it was feeling a little **cheesy**!', 'explanation': 'This joke is a classic pun, playing on the double meaning of the word **"cheesy"**.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Obviously, pizza is made with cheese. So, in a literal sense, a pizza is *always* "cheesy" because it contains cheese. This is the setup for the joke.\n\n2.  **Figurative Meaning (Feelings/Behavior):** The word "cheesy" is also used to describe something that is overly sentimental, sappy, corny, or trying too hard to be romantic or emotional. When someone expresses their "deepest feelings" in an overly dramatic or sentimental way, we often say it\'s "cheesy" (e.g., "That\'s a cheesy pick-up line," or "That love song is a bit cheesy").\n\nThe humor comes from the unexpected link between these two meanings. The pizza, being literally full of cheese, is also metaphorically "cheesy" in i

In [9]:
list(workflow.get_state_history(config1)) # to get intermediate state values

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza confess its deepest feelings?\n\nBecause it was feeling a little **cheesy**!', 'explanation': 'This joke is a classic pun, playing on the double meaning of the word **"cheesy"**.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Obviously, pizza is made with cheese. So, in a literal sense, a pizza is *always* "cheesy" because it contains cheese. This is the setup for the joke.\n\n2.  **Figurative Meaning (Feelings/Behavior):** The word "cheesy" is also used to describe something that is overly sentimental, sappy, corny, or trying too hard to be romantic or emotional. When someone expresses their "deepest feelings" in an overly dramatic or sentimental way, we often say it\'s "cheesy" (e.g., "That\'s a cheesy pick-up line," or "That love song is a bit cheesy").\n\nThe humor comes from the unexpected link between these two meanings. The pizza, being literally full of cheese, is also metaphorically "cheesy" in 

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was tired of being *sauced* around!',
 'explanation': 'This joke is a classic **pun**, relying on a play on words!\n\nHere\'s the breakdown:\n\n1.  **The Setup:** Pasta and sauce are natural companions. The idea of them "breaking up" is anthropomorphizing them (giving human qualities to inanimate objects), which is common in jokes.\n\n2.  **The Pun:** The punchline "sauced around" is the key.\n    *   It literally refers to the pasta being mixed or covered in **sauce**.\n    *   However, it sounds almost exactly like the phrase "**bossed around**."\n\n3.  **The Meaning of "Bossed Around":** To be "bossed around" means to be told what to do, controlled, pushed around, or treated unfairly by someone in authority.\n\n**Therefore, the humor comes from:**\n\nThe joke implies the pasta feels like it\'s being manipulated or controlled, just like a person who is "bossed around." It\'s a silly, relatable reaso

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza confess its deepest feelings?\n\nBecause it was feeling a little **cheesy**!', 'explanation': 'This joke is a classic pun, playing on the double meaning of the word **"cheesy"**.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning (Pizza):** Obviously, pizza is made with cheese. So, in a literal sense, a pizza is *always* "cheesy" because it contains cheese. This is the setup for the joke.\n\n2.  **Figurative Meaning (Feelings/Behavior):** The word "cheesy" is also used to describe something that is overly sentimental, sappy, corny, or trying too hard to be romantic or emotional. When someone expresses their "deepest feelings" in an overly dramatic or sentimental way, we often say it\'s "cheesy" (e.g., "That\'s a cheesy pick-up line," or "That love song is a bit cheesy").\n\nThe humor comes from the unexpected link between these two meanings. The pizza, being literally full of cheese, is also metaphorically "cheesy" in i

In [12]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was tired of being *sauced* around!', 'explanation': 'This joke is a classic **pun**, relying on a play on words!\n\nHere\'s the breakdown:\n\n1.  **The Setup:** Pasta and sauce are natural companions. The idea of them "breaking up" is anthropomorphizing them (giving human qualities to inanimate objects), which is common in jokes.\n\n2.  **The Pun:** The punchline "sauced around" is the key.\n    *   It literally refers to the pasta being mixed or covered in **sauce**.\n    *   However, it sounds almost exactly like the phrase "**bossed around**."\n\n3.  **The Meaning of "Bossed Around":** To be "bossed around" means to be told what to do, controlled, pushed around, or treated unfairly by someone in authority.\n\n**Therefore, the humor comes from:**\n\nThe joke implies the pasta feels like it\'s being manipulated or controlled, just like a person who is "bossed around." It\'s a si

## Time Travel

### It is used for debugging

In [35]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0a6191-4e07-6d3e-8000-c91a35f0a302"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0a6191-4e07-6d3e-8000-c91a35f0a302'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-10-10T20:38:30.313457+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a6191-4dde-6ea1-bfff-7e79b0dd7e17'}}, tasks=(PregelTask(id='63a2bf69-859e-d003-2b28-9f36c547a232', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza confess its deepest feelings?\n\nBecause it was feeling a little **cheesy**!'}),), interrupts=())

In [36]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0a6191-4e07-6d3e-8000-c91a35f0a302"}})

{'topic': 'pizza',
 'joke': "Why did the pizza chef go broke?\n\nBecause he didn't have enough **dough**!",
 'explanation': 'This joke is a classic example of a **pun**, which relies on a word having two different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for a pizza chef):** For a pizza chef, "dough" is the essential mixture of flour, water, yeast, etc., that they use to make the crust of the pizza. It\'s their primary ingredient.\n\n2.  **"Dough" (Meaning 2 - Slang):** "Dough" is also a very common, informal slang term for **money**. If you don\'t have enough "dough," you don\'t have enough money.\n\n**The Joke:**\n\nThe joke sets you up to think about a pizza chef\'s business in terms of ingredients. When you hear "Why did the pizza chef go broke?", your mind might first go to things like not enough customers, bad recipes, or maybe running out of *pizza dough*.\n\nHowever, the punchline cleverly uses the *other* meaning of "dough" to explain his finan

In [37]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza chef go broke?\n\nBecause he didn't have enough **dough**!", 'explanation': 'This joke is a classic example of a **pun**, which relies on a word having two different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for a pizza chef):** For a pizza chef, "dough" is the essential mixture of flour, water, yeast, etc., that they use to make the crust of the pizza. It\'s their primary ingredient.\n\n2.  **"Dough" (Meaning 2 - Slang):** "Dough" is also a very common, informal slang term for **money**. If you don\'t have enough "dough," you don\'t have enough money.\n\n**The Joke:**\n\nThe joke sets you up to think about a pizza chef\'s business in terms of ingredients. When you hear "Why did the pizza chef go broke?", your mind might first go to things like not enough customers, bad recipes, or maybe running out of *pizza dough*.\n\nHowever, the punchline cleverly uses the *other* meaning of "dough" 

## Updating state

In [38]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0a6191-4e07-6d3e-8000-c91a35f0a302", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0a61dc-62d4-6086-8001-db3e2fd88396'}}

In [39]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a61dc-62d4-6086-8001-db3e2fd88396'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-10-10T21:12:05.760144+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a6191-4e07-6d3e-8000-c91a35f0a302'}}, tasks=(PregelTask(id='1d72fa63-2c1d-b767-be23-ba6408ecf64f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza chef go broke?\n\nBecause he didn't have enough **dough**!", 'explanation': 'This joke is a classic example of a **pun**, which relies on a word having two different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Dough" (Meaning 1 - Literal for a pizza chef):** For a pizza chef, "dough" is the essential mixture 

In [41]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0a61dc-62d4-6086-8001-db3e2fd88396"}})

{'topic': 'samosa',
 'joke': 'Here are a few samosa jokes for you:\n\n1.  Why did the samosa break up with the pakora?\n    Because it said, "You just don\'t **spice** things up enough for me!"\n\n2.  What\'s a samosa\'s favorite day of the week?\n    **Fry-day!**\n\n3.  Why are samosas so good at geometry?\n    Because they always have all their **angles** covered!\n\n4.  What do you call a lazy samosa?\n    A **slow-mosa!**\n\n5.  I told my friend I was craving a samosa. He said, "You mean **some-oh-sa** good?"',
 'explanation': 'These are all puns, playing on words that sound alike or have multiple meanings! Here\'s an explanation for each:\n\n1.  **Why did the samosa break up with the pakora? Because it said, "You just don\'t spice things up enough for me!"**\n    *   **Explanation:** This joke plays on two meanings of "spice."\n        *   Literally, **samosas** are often filled with spicy ingredients, so they *have* a lot of spice. Pakoras can be less spicy.\n        *   Figurati

In [42]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Here are a few samosa jokes for you:\n\n1.  Why did the samosa break up with the pakora?\n    Because it said, "You just don\'t **spice** things up enough for me!"\n\n2.  What\'s a samosa\'s favorite day of the week?\n    **Fry-day!**\n\n3.  Why are samosas so good at geometry?\n    Because they always have all their **angles** covered!\n\n4.  What do you call a lazy samosa?\n    A **slow-mosa!**\n\n5.  I told my friend I was craving a samosa. He said, "You mean **some-oh-sa** good?"', 'explanation': 'These are all puns, playing on words that sound alike or have multiple meanings! Here\'s an explanation for each:\n\n1.  **Why did the samosa break up with the pakora? Because it said, "You just don\'t spice things up enough for me!"**\n    *   **Explanation:** This joke plays on two meanings of "spice."\n        *   Literally, **samosas** are often filled with spicy ingredients, so they *have* a lot of spice. Pakoras can be less spicy.\n

## Fault Tolerance

In [23]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [24]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str

In [25]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(100)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [26]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [27]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [28]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [29]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a61c7-4922-6099-8003-080ed33bc807'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-10-10T21:02:39.351351+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a61c7-490e-6148-8002-feb4baf4e5a3'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a61c7-490e-6148-8002-feb4baf4e5a3'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-10T21:02:39.343134+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a61c3-3ceb-6848-8001-6b11f456e1b8'}}, tasks=(PregelTask(id='4bfc31c5-aa43-b57f-dbb0-a5044b17c7e